<a href="https://colab.research.google.com/github/JulieMew/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/MMA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

 Choose your target. Which column in your tabular dataset will you predict?

 method_d

Is your problem regression or classification?


 Regression.

How is your target distributed?
Regression: Is the target right-skewed? If so, you may want to log transform the target.

Choose your evaluation metric(s).
Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?

Precision-Recall AUC,
F1-Measure,
Accuracy


Choose which observations you will use to train, validate, and test your model.
Split all of the method_d into unique categories and use those.

Are some observations outliers? Will you exclude them?
I don't think any are outliers yet.
Will you do a random split or a time-based split?
time-based split

Begin to clean and explore your data.

Begin to choose which features, if any, to exclude. Would some features "leak" future information?

Begin to clean and explore your data.

In [2]:
# Begin PCA
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [3]:
# Read MMA Data 
from google.colab import files
uploaded = files.upload()

Saving ufc_fights.csv to ufc_fights.csv


In [9]:
# Read MMA Data 
from google.colab import files
uploaded = files.upload()

Saving ufc_fighters.csv to ufc_fighters.csv


In [10]:
# read csv
df = pd.read_csv('ufc_fights.csv')

print(df.shape)
df.head()

(3569, 20)


,pageurl,eid,mid,event_name,event_org,event_date,event_place,f1pageurl,f2pageurl,f1name,f2name,f1result,f2result,f1fid,f2fid,method,method_d,ref,round,time
0,/events/UFC-1-The-Beginning-7,7,8,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Royce-Gracie-19,/fighter/Gerard-Gordeau-15,Royce Gracie,Gerard Gordeau,win,loss,19,15,Submission,Rear-Naked Choke,Helio Vigio,1,1:44
1,/events/UFC-1-The-Beginning-7,7,7,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Jason-DeLucia-22,/fighter/Trent-Jenkins-23,Jason DeLucia,Trent Jenkins,win,loss,22,23,Submission,Rear-Naked Choke,Joao Alberto Barreto,1,0:52
2,/events/UFC-1-The-Beginning-7,7,6,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Royce-Gracie-19,/fighter/Ken-Shamrock-4,Royce Gracie,Ken Shamrock,win,loss,19,4,Submission,Rear-Naked Choke,Helio Vigio,1,0:57
3,/events/UFC-1-The-Beginning-7,7,5,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Gerard-Gordeau-15,/fighter/Kevin-Rosier-17,Gerard Gordeau,Kevin Rosier,win,loss,15,17,TKO,Corner Stoppage,Joao Alberto Barreto,1,0:59
4,/events/UFC-1-The-Beginning-7,7,4,UFC 1 - The Beginning,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",/fighter/Ken-Shamrock-4,/fighter/Patrick-Smith-21,Ken Shamrock,Patrick Smith,win,loss,4,21,Submission,Heel Hook,Helio Vigio,1,1:49


In [11]:
# read csv
fighters = pd.read_csv('ufc_fighters.csv')

print(df.shape)
fighters.head()

(3569, 20)


,url,fid,name,nick,birth_date,height,weight,association,class,locality,country
0,/fighter/Conor-McGregor-29688,29688,Conor McGregor,Notorious,7/14/1988,68.0,145.0,SBG Ireland,Featherweight,Dublin,Ireland
1,/fighter/Jon-Jones-27944,27944,Jon Jones,Bones,7/19/1987,76.0,205.0,Jackson-Wink MMA,Light Heavyweight,"Rochester, New York",United States
2,/fighter/Holly-Holm-75125,75125,Holly Holm,The Preacher's Daughter,10/17/1981,68.0,135.0,Jackson-Wink MMA,Bantamweight,"Albuquerque, New Mexico",United States
3,/fighter/Dominick-Cruz-12107,12107,Dominick Cruz,The Dominator,9/3/1985,68.0,134.0,Alliance MMA,Bantamweight,"San Diego, California",United States
4,/fighter/Demetrious-Johnson-45452,45452,Demetrious Johnson,Mighty Mouse,8/13/1986,63.0,125.0,AMC Pankration,Flyweight,"Kirkland, Washington",United States


Remove all useless features.

In [12]:
# make a variable for how many total wins a fighter has.

wins = df.groupby('f1result').apply(lambda x: x['f1fid'].nunique())
wins.head()


f1result
NC       35
draw     20
win     974
dtype: int64

In [13]:
# make dataframe with features f1fid for the ID number of each fighter. This will make it easier to generate
# hypothetical fighters for possible scenarios.
#f for features
f = df['f1fid'].value_counts()
f.head()

3500     19
232      18
10196    17
5993     17
1356     17
Name: f1fid, dtype: int64

In [14]:
#make a dataframe with method_d 
#t for target
t = df['method_d'].value_counts()
t.head()

Unanimous           1142
Punches              699
Split                308
Rear-Naked Choke     252
Guillotine Choke     134
Name: method_d, dtype: int64

In [15]:
df = df.drop(["pageurl", "eid", 'f1result', "mid", 'event_name', 'event_org', 'event_date', 'event_place', 'f2pageurl', 'f1name', 'f2name', 'f2result', 'f2fid', 'method', 'ref', 'round', 'time'], axis=1)

In [16]:
df = df.drop(["f1pageurl"], axis=1)

In [17]:
pd.factorize(df['method_d'])

(array([ 0,  0,  0, ..., 25, 99, 24]),
 Index(['Rear-Naked Choke', 'Corner Stoppage', 'Heel Hook', 'Smother Choke',
        'Punches', 'Head Kick', 'Lapel Choke', 'Guillotine Choke', 'Armlock',
        'Elbows',
        ...
        'Finger Injury', 'Overturned by NAC', 'Elbow and Punch',
        'Overturned by CABMMA', 'Hook Kick and Punches', 'Esophagus Injury',
        'Hand Injury', 'Body Kick and Punches', 'Japanese Necktie',
        'Elbows and Knees'],
       dtype='object', length=175))

In [18]:
df['method_d'].unique()

array(['Rear-Naked Choke', 'Corner Stoppage', 'Heel Hook',
       'Smother Choke', 'Punches', 'Head Kick', 'Lapel Choke',
       'Guillotine Choke', 'Armlock', 'Elbows', 'Knees', 'Punch',
       'Triangle Choke', 'Rear Naked Choke', 'Choke', nan, 'Armbar',
       'Keylock', 'Cut', 'Front Choke', 'Elbow', 'Strikes', 'Position',
       'Kneebar', 'Doctor Stoppage', 'Unanimous', 'Split',
       'Doctor Stoppage - Broken Nose', 'Exhaustion', 'Neck Crank',
       'Forearm Choke', 'Fatigue', 'Headbutts', 'Achilles Lock',
       'Crucifix Choke', 'Chin to the Eye', 'Knee', 'Slam',
       'Shoulder Choke', 'Submission (Armbar',
       'Submission (Arm-Triangle Choke', 'Arm-Triangle Choke', 'Majority',
       'Kicking with Shoes', 'Punches and Elbows', 'Triangle Armbar',
       'Dislocated Shoulder', 'Bulldog Choke', 'Spinning Back Fist',
       'Kick to the Body and Punches', 'Repeated Fouls',
       'Knee and Punches', 'Inside Shoulder Lock', 'Injury', 'Kimura',
       'Toe Hold', 'Knees and 

In [19]:
# make method_d numeric


In [20]:
#dataframe with only important variables
df

,f1fid,method_d
0,19,Rear-Naked Choke
1,22,Rear-Naked Choke
2,19,Rear-Naked Choke
3,15,Corner Stoppage
4,4,Heel Hook
...,...,...
3564,29470,Unanimous
3565,25412,Unanimous
3566,75021,Split
3567,66725,Elbows and Punches


In [21]:
# Go through unique values and make them STR to INt
df['method_d'].unique()[143]

'Spinning Hook Kick'

In [22]:
# Go through values out of curiosity
df.loc[df['method_d'] == 145]

,f1fid,method_d


In [23]:
df.method_d[df.method_d == 'Rear-Naked Choke'] = 1
df.method_d[df.method_d == 'Corner Stoppage'] = 2
df.method_d[df.method_d == 'Heel Hook'] = 3
df.method_d[df.method_d == 'Smother Choke'] = 4
df.method_d[df.method_d == 'Punches'] = 5
df.method_d[df.method_d == 'Head Kick'] = 6
df.method_d[df.method_d == 'Lapel Choke'] = 7
df.method_d[df.method_d == 'Guillotine Choke'] = 8
df.method_d[df.method_d == 'Armlock'] = 9
df.method_d[df.method_d == 'Elbows'] = 10
df.method_d[df.method_d == 'Knees'] = 11
df.method_d[df.method_d == 'Punch'] = 12
df.method_d[df.method_d == 'Triangle Choke'] = 13
df.method_d[df.method_d == 'Rear Naked Choke'] = 14
df.method_d[df.method_d == 'Choke'] = 15
df.method_d[df.method_d == 'Armbar'] = 16
df.method_d[df.method_d == 'Keylock'] = 17
df.method_d[df.method_d == 'Cut'] = 18
df.method_d[df.method_d == 'Front Choke'] = 19
df.method_d[df.method_d == 'Elbow'] = 20
df.method_d[df.method_d == 'Strikes'] = 21
df.method_d[df.method_d == 'Position'] = 22
df.method_d[df.method_d == 'Kneebar'] = 23
df.method_d[df.method_d == 'Doctor Stoppage'] = 24
df.method_d[df.method_d == 'Unanimous'] = 25
df.method_d[df.method_d == 'Split'] = 26
df.method_d[df.method_d == 'Doctor Stoppage - Broken Nose'] = 27
df.method_d[df.method_d == 'Exhaustion'] = 28
df.method_d[df.method_d == 'Neck Crank'] = 29
df.method_d[df.method_d == 'Forearm Choke'] = 30
df.method_d[df.method_d == 'Fatigue'] = 31
df.method_d[df.method_d == 'Headbutts'] = 32
df.method_d[df.method_d == 'Achilles Lock'] = 33
df.method_d[df.method_d == 'Crucifix Choke'] = 34
df.method_d[df.method_d == 'Chin to the Eye'] = 35 
df.method_d[df.method_d == 'Knee'] = 36
df.method_d[df.method_d == 'Shoulder Choke'] = 37
df.method_d[df.method_d == 'Submission (Armbar'] = 38
df.method_d[df.method_d == 'Shoulder Choke'] = 39
df.method_d[df.method_d == 'Submission (Arm-Triangle Choke'] = 40
df.method_d[df.method_d == 'Arm-Triangle Choke'] = 42
df.method_d[df.method_d == 'Majority'] = 43
df.method_d[df.method_d == 'Kicking with Shoes'] = 44
df.method_d[df.method_d == 'Punches and Elbows'] = 45
df.method_d[df.method_d == 'Triangle Armbar'] = 46
df.method_d[df.method_d == 'Dislocated Shoulder'] = 47
df.method_d[df.method_d == 'Bulldog Choke'] = 48
df.method_d[df.method_d == 'Spinning Back Fist'] = 49
df.method_d[df.method_d == 'Kick to the Body and Punches'] = 50
df.method_d[df.method_d == 'Repeated Fouls'] = 51
df.method_d[df.method_d == 'Knee and Punches'] = 52
df.method_d[df.method_d == 'Inside Shoulder Lock'] = 53
df.method_d[df.method_d == 'Injury'] = 54
df.method_d[df.method_d == 'Toe Hold'] = 55
df.method_d[df.method_d == 'Arm Triangle Choke'] = 56
df.method_d[df.method_d == 'Punches and Knees'] = 57
df.method_d[df.method_d == 'Flying Head Kick and Punches'] = 58
df.method_d[df.method_d == 'Body Lock'] = 59
df.method_d[df.method_d == 'Spinning Back Kick'] = 60
df.method_d[df.method_d == 'Submission (Rear-Naked Choke'] = 61
df.method_d[df.method_d == 'Retirement'] = 62
df.method_d[df.method_d == 'Anaconda Choke'] = 63
df.method_d[df.method_d == 'Cuts'] = 64
df.method_d[df.method_d == 'Contest (Sakara Kicked in Groin'] = 65
df.method_d[df.method_d == 'Leg Kicks'] = 66
df.method_d[df.method_d == 'Superman Punch'] = 67
df.method_d[df.method_d == 'Submission (Shoulder Choke'] = 68
df.method_d[df.method_d == 'Head Kick and Knee'] = 69
df.method_d[df.method_d == 'Submission (North-South Choke'] = 70
df.method_d[df.method_d == 'Upkick and Punches'] = 71
df.method_d[df.method_d == 'Punch to the Body'] = 72
df.method_d[df.method_d == 'Neck Injury'] = 73
df.method_d[df.method_d == 'Kick to the Body and Elbows'] = 74
df.method_d[df.method_d == 'Brabo Choke'] = 75
df.method_d[df.method_d == 'Suplex and Punches'] = 76
df.method_d[df.method_d == 'Knee Injury'] = 77
df.method_d[df.method_d == 'Shoulder Injury'] = 78
df.method_d[df.method_d == 'Triangle Kimura'] = 79
df.method_d[df.method_d == 'Punch and Knee'] = 80
df.method_d[df.method_d == 'Illegal Knee to the Head'] = 81
df.method_d[df.method_d == 'Slam and Punches'] = 82
df.method_d[df.method_d == 'Knees to the Body'] = 83
df.method_d[df.method_d == 'Arm Injury'] = 84
df.method_d[df.method_d == 'Submission (Triangle Choke'] = 85
df.method_d[df.method_d == 'Leg Injury'] = 86
df.method_d[df.method_d == 'Submission (Anaconda Choke'] = 87
df.method_d[df.method_d == 'Flying Knee and Punches'] = 88
df.method_d[df.method_d == 'Eye Injury'] = 89
df.method_d[df.method_d == 'Peruvian Necktie'] = 90
df.method_d[df.method_d == 'Kick to the Body'] = 91
df.method_d[df.method_d == 'Reverse Armbar'] = 92
df.method_d[df.method_d == 'Rib Injury'] = 93
df.method_d[df.method_d == 'Elbows and Punches'] = 94
df.method_d[df.method_d == 'Spinning Back Kick and Punches'] = 95
df.method_d[df.method_d == 'Decision (Overturned by NSAC'] = 96
df.method_d[df.method_d == 'Submission (Guillotine Choke'] = 97
df.method_d[df.method_d == 'Illegal Downward Elbows'] = 98
df.method_d[df.method_d == 'Submission (Brabo Choke'] = 75
df.method_d[df.method_d == 'Elbows to the Body'] = 99
df.method_d[df.method_d == 'Ankle Injury'] = 100
df.method_d[df.method_d == 'Illegal Upkick'] = 101
df.method_d[df.method_d == 'Knee to the Body'] = 102
df.method_d[df.method_d == 'Jaw Injury'] = 103
df.method_d[df.method_d == 'Pillory Choke'] = 104
df.method_d[df.method_d == 'Overturned by NSAC'] = 105
df.method_d[df.method_d == 'Body Triangle'] = 106
df.method_d[df.method_d == 'Front Kick and Punches'] = 107
df.method_d[df.method_d == 'Knee to the Body and Punches'] = 108
df.method_d[df.method_d == 'Twister'] = 109
df.method_d[df.method_d == 'Front Kick'] = 110
df.method_d[df.method_d == 'Elbow and Punches'] = 111
df.method_d[df.method_d == 'Flying Triangle Choke'] = 112
df.method_d[df.method_d == 'Lentz Struck by Illegal Knee'] = 81
df.method_d[df.method_d == 'Inverted Heel Hook'] = 113
df.method_d[df.method_d == 'Accidental Head-butt'] = 114
df.method_d[df.method_d == 'Knees to the Body and Punches'] = 115
df.method_d[df.method_d == 'Submission (Kimura'] = 116
df.method_d[df.method_d == 'Punches and Elbows to the Body'] = 117
df.method_d[df.method_d == 'Punches to Back of Head'] = 118
df.method_d[df.method_d == 'Spinning Wheel Kick'] = 119
df.method_d[df.method_d == 'Calf Slicer'] = 120
df.method_d[df.method_d == 'Punch and Elbows'] = 121
df.method_d[df.method_d == 'Overturned by Commission'] = 122
df.method_d[df.method_d == 'Accidental Eye Poke'] = 133
df.method_d[df.method_d == 'Head Kick and Punch'] = 134
df.method_d[df.method_d == 'Contest (Overturned by Promoter'] = 135
df.method_d[df.method_d == 'Overturned by Promoter'] = 136
df.method_d[df.method_d == 'Accidental Foul'] = 137
df.method_d[df.method_d == 'Knees and Elbow'] = 138
df.method_d[df.method_d == 'Decision (Unanimous'] = 25
df.method_d[df.method_d == 'Thumb Injury'] = 139
df.method_d[df.method_d == 'Decision (Majority'] = 140
df.method_d[df.method_d == 'Knee to Head of Grounded Fighter'] = 141
df.method_d[df.method_d == 'Spinning Back-Kick and Punches'] = 142
df.method_d[df.method_d == 'Slam and Punch'] = 143
df.method_d[df.method_d == 'Decision (Overturned by TDLR'] = 144
df.method_d[df.method_d == 'Elbows to Back of Head'] = 145
df.method_d[df.method_d == 'Wheel Kick and Punches'] = 146
df.method_d[df.method_d == 'Spinning Back Elbow'] = 147
df.method_d[df.method_d == 'Leg Kick'] = 148
df.method_d[df.method_d == 'Punches and Knee'] = 149
df.method_d[df.method_d == 'Accidental Clash of Heads'] = 150
df.method_d[df.method_d == 'Inverted Triangle Kimura'] = 151
df.method_d[df.method_d == 'Submission to Heel Strikes'] = 152
df.method_d[df.method_d == 'Overturned by Nevada Athletic Commission'] = 153
df.method_d[df.method_d == 'Contest (Overturned by Commission'] = 154
df.method_d[df.method_d == 'Overturned by CSAC'] = 155
df.method_d[df.method_d == 'Omoplata'] = 156
df.method_d[df.method_d == 'Reverse Bulldog Choke'] = 157
df.method_d[df.method_d == 'Punches to the Body'] = 158
df.method_d[df.method_d == 'Overturned by NSBA'] = 159
df.method_d[df.method_d == 'Side Kick and Punches'] = 160
df.method_d[df.method_d == 'Submission (Bulldog Choke'] = 161
df.method_d[df.method_d == 'Finger Injury'] = 162
df.method_d[df.method_d == 'Overturned by NAC'] = 163
df.method_d[df.method_d == 'Elbow and Punch'] = 164
df.method_d[df.method_d == 'Overturned by CABMMA'] = 165
df.method_d[df.method_d == 'Hook Kick and Punches'] = 166
df.method_d[df.method_d == 'Esophagus Injury'] = 167
df.method_d[df.method_d == 'Hand Injury'] = 168
df.method_d[df.method_d == 'Body Kick and Punches'] = 169
df.method_d[df.method_d == 'Japanese Necktie'] = 170
df.method_d[df.method_d == 'Elbows and Knees'] = 171
df.method_d[df.method_d == 'Reverse Triangle Armbar'] = 172
df.method_d[df.method_d == 'Slam'] = 173

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [24]:
df

,f1fid,method_d
0,19,1
1,22,1
2,19,1
3,15,2
4,4,3
...,...,...
3564,29470,25
3565,25412,25
3566,75021,26
3567,66725,94


#Split the data

In [25]:
train = df.iloc[:, :1784]

test = df.iloc[:, 1784:]

#Preprocessing

In [35]:
import pandas 
import scipy 
import numpy 
from sklearn.preprocessing import MinMaxScaler 
names = ['f1fid', 'method_d'] 
array = train.values

# separate array into input and output components 
X = array[:,0:2] 
Y = array[:,1] 
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.__init__(X).all
rescaledX = scaler.fit(X)  

# summarize transformed data 
numpy.set_printoptions(precision=3)
rescaledX = sorted(rescaledX)
rescaledX[0:5,:]

TypeError: ignored

Modeling

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 1, 1, 0])
y_scores = np.array([1, 0, 0, 0])
roc_auc_score(y_true, y_scores)

We can see data leakage of method d. Data leakage is a situation where information from outside the training dataset is used to create the model. This additional information can allow the model to learn or know something that it otherwise wouldn't know, which ruins the effectiveness of the model.